# CSDI for Time Series Forecasting

This notebook demonstrates the usage of the CSDI (Conditional Score-based Diffusion Models) model for time series forecasting tasks, specifically for electricity consumption data.

In [2]:
!pip install gdown

  Obtaining dependency information for gdown from https://files.pythonhosted.org/packages/54/70/e07c381e6488a77094f04c85c9caf1c8008cdc30778f7019bc52e5285ef0/gdown-5.2.0-py3-none-any.whl.metadata


## Library Imports and Model Setup

In this section, we import necessary libraries and modules required for the implementation of the CSDI model. This includes standard data handling libraries like `numpy` and `pandas`, deep learning libraries from `torch`, and specific components for building and training the CSDI model.

In [20]:
import os
import sys
import argparse
import torch.nn as nn
import yaml
import numpy as np
import datetime
import json
import gdown
import zipfile

# Add the parent directory to sys.path to import local modules
sys.path.append(os.path.abspath(os.path.join('.')))

#from main_model import CSDI_Forecasting
from dataset_forecasting import get_dataloader
from utils import train, evaluate
from diff_models import diff_CSDI

## Download Dataset

Run this cell to download the dataset if you haven't already.

In [13]:
def download_data(folder_url, output_path='data/electricity_nips'):
    """
    Download all files from a Google Drive folder and save them to the specified output path.
    Checks if data already exists in the output directory before downloading.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    # Check if the directory is not empty, implying data might already be downloaded
    if os.listdir(output_path):
        print(f"Data already exists in {output_path}. Skipping download.")
        return
    
    # Extract the folder ID from the URL
    folder_id = folder_url.split('/')[-1]
    
    # List files in the folder
    url = f"https://drive.google.com/drive/folders/{folder_id}"
    output = gdown.download_folder(url, output=output_path, quiet=False, use_cookies=False)
    
    if output is None:
        print("Failed to download files. Make sure the folder is publicly accessible.")
    else:
        print(f"Downloaded files to {output_path}")

# Example usage with a specified folder URL
folder_url = "https://drive.google.com/drive/folders/1krZQofLdeQrzunuKkLXy8L_kMzQrVFI_"
download_data(folder_url)


Data already exists in data. Skipping download.


In [14]:
class CSDI_base(nn.Module):
    """
    The `CSDI_base` class is a PyTorch module that facilitates Conditional Score-based Diffusion Modeling for 
    time series.

    Key Components and Initializations:

    - `__init__`: Constructor that initializes the model with:
      - `target_dim`: The dimensionality of the target data, dictating the model's output features.
      - `config`: Configuration settings for model parameters such as embeddings and the diffusion process.
      - `device`: Computation device (CPU or GPU) for tensor operations, ensuring optimal performance.

    Configuration and Setup:

    1. Embedding Dimensions:
       - `emb_time_dim` and `emb_feature_dim`: Define embedding sizes for time points and features, 
          transforming input data for the network.
       - `is_unconditional`: Adjusts embedding dimensions based on whether the model is conditional, 
          integrating observed data when necessary.

    2. Network Layers:
       - `embed_layer`: A PyTorch embedding layer that maps features to dense vectors, enhancing data pattern recognition.

    3. Diffusion Model Configuration:
       - Extracts diffusion-related settings from `config`, like the number of diffusion steps and the beta schedule, 
         which dictate the noise addition process.

    4. Diffusion Parameters:
       - `beta`, `alpha_hat`, `alpha`: Arrays that manage the noise levels and data transformation during the diffusion process.
       - `alpha_torch`: Converts the `alpha` array to a PyTorch tensor formatted for use in the model's operations and 
          placed on the designated device.
    """
    def __init__(self, target_dim, config, device):
        super().__init__()
        self.device = device
        self.target_dim = target_dim

        self.emb_time_dim = config["model"]["timeemb"]
        self.emb_feature_dim = config["model"]["featureemb"]
        self.is_unconditional = config["model"]["is_unconditional"]
        self.target_strategy = config["model"]["target_strategy"]

        self.emb_total_dim = self.emb_time_dim + self.emb_feature_dim
        if self.is_unconditional == False:
            self.emb_total_dim += 1  # for conditional mask
        self.embed_layer = nn.Embedding(
            num_embeddings=self.target_dim, embedding_dim=self.emb_feature_dim
        )

        config_diff = config["diffusion"]
        config_diff["side_dim"] = self.emb_total_dim

        input_dim = 1 if self.is_unconditional == True else 2
        self.diffmodel = diff_CSDI(config_diff, input_dim)

        # parameters for diffusion models
        self.num_steps = config_diff["num_steps"]
        if config_diff["schedule"] == "quad":
            self.beta = np.linspace(
                config_diff["beta_start"] ** 0.5, config_diff["beta_end"] ** 0.5, self.num_steps
            ) ** 2
        elif config_diff["schedule"] == "linear":
            self.beta = np.linspace(
                config_diff["beta_start"], config_diff["beta_end"], self.num_steps
            )

        self.alpha_hat = 1 - self.beta
        self.alpha = np.cumprod(self.alpha_hat)
        self.alpha_torch = torch.tensor(self.alpha).float().to(self.device).unsqueeze(1).unsqueeze(1)

    """
    def time_embedding
       - Constructs a time-based embedding for input positions using sinusoidal functions, commonly used in models like transformers.
       - `pos`: Tensor of positions for which embeddings are generated.
       - `d_model`: Dimensionality of the embedding; defaults to 128.
       - The function uses sine and cosine functions to embed time positions, which helps in capturing periodic patterns.
    """
    def time_embedding(self, pos, d_model=128):
        pe = torch.zeros(pos.shape[0], pos.shape[1], d_model).to(self.device)
        position = pos.unsqueeze(2)
        div_term = 1 / torch.pow(
            10000.0, torch.arange(0, d_model, 2).to(self.device) / d_model
        )
        pe[:, :, 0::2] = torch.sin(position * div_term)
        pe[:, :, 1::2] = torch.cos(position * div_term)
        return pe

    """
    def get_randmask
       - Generates a random mask for observed data points to simulate missingness during training, 
         enhancing the model's robustness.
       - `observed_mask`: A tensor indicating which data points are observed.
       - This function randomly masks observed data points based on their existing observed status, 
         useful in self-supervised setups where true missing patterns need to be approximated.
    """
    def get_randmask(self, observed_mask):
        rand_for_mask = torch.rand_like(observed_mask) * observed_mask
        rand_for_mask = rand_for_mask.reshape(len(rand_for_mask), -1)
        for i in range(len(observed_mask)):
            sample_ratio = np.random.rand()  # missing ratio
            num_observed = observed_mask[i].sum().item()
            num_masked = round(num_observed * sample_ratio)
            rand_for_mask[i][rand_for_mask[i].topk(num_masked).indices] = -1
        cond_mask = (rand_for_mask > 0).reshape(observed_mask.shape).float()
        return cond_mask

    """
    def get_hist_mask
       - Generates a historical mask based on the observed data and possibly an external pattern.
       - `observed_mask`: A tensor indicating observed data points.
       - `for_pattern_mask`: An optional tensor to provide historical patterns for masking, defaults to the observed_mask.
       - Depending on the `target_strategy`, it combines random masking and historical patterns to simulate different training conditions.

    """
    def get_hist_mask(self, observed_mask, for_pattern_mask=None):
        if for_pattern_mask is None:
            for_pattern_mask = observed_mask
        if self.target_strategy == "mix":
            rand_mask = self.get_randmask(observed_mask)

        cond_mask = observed_mask.clone()
        for i in range(len(cond_mask)):
            mask_choice = np.random.rand()
            if self.target_strategy == "mix" and mask_choice > 0.5:
                cond_mask[i] = rand_mask[i]
            else:  # draw another sample for histmask (i-1 corresponds to another sample)
                cond_mask[i] = cond_mask[i] * for_pattern_mask[i - 1] 
        return cond_mask
    
    """
    def get_test_pattern_mask
       - Produces a test pattern mask that is the element-wise product of observed data points and a specified test pattern.
       - `observed_mask`: A tensor of observed data points.
       - `test_pattern_mask`: A predefined tensor describing specific test conditions or missing data patterns.
       - This function is useful for simulating specific scenarios during model evaluation or testing.
    """
    def get_test_pattern_mask(self, observed_mask, test_pattern_mask):
        return observed_mask * test_pattern_mask

    """
    def get_side_info
       - Constructs side information by combining time and feature embeddings with an optional conditional mask.
       - `observed_tp`: Tensor of observed time points.
       - `cond_mask`: A condition mask indicating available data points.
       - This function enriches the model input with necessary contextual information, adjusting for the presence of observed data 
         and enhancing the model's ability to interpret and impute based on time and feature dependencies.
    """
    def get_side_info(self, observed_tp, cond_mask):
        B, K, L = cond_mask.shape

        time_embed = self.time_embedding(observed_tp, self.emb_time_dim)  # (B,L,emb)
        time_embed = time_embed.unsqueeze(2).expand(-1, -1, K, -1)
        feature_embed = self.embed_layer(
            torch.arange(self.target_dim).to(self.device)
        )  # (K,emb)
        feature_embed = feature_embed.unsqueeze(0).unsqueeze(0).expand(B, L, -1, -1)

        side_info = torch.cat([time_embed, feature_embed], dim=-1)  # (B,L,K,*)
        side_info = side_info.permute(0, 3, 2, 1)  # (B,*,K,L)

        if self.is_unconditional == False:
            side_mask = cond_mask.unsqueeze(1)  # (B,1,K,L)
            side_info = torch.cat([side_info, side_mask], dim=1)

        return side_info

    """
    def calc_loss_valid
       - Computes the validation loss by aggregating losses over all diffusion steps.
       - `observed_data`: Tensor containing the observed data points.
       - `cond_mask`, `observed_mask`: Masks indicating conditions and observed data points.
       - `side_info`: Additional contextual information provided to the diffusion model.
       - `is_train`: A flag indicating whether the function is being called in training mode.
       - Iterates over all diffusion steps, accumulating loss calculated at each step and averages it.
    """
    def calc_loss_valid(
        self, observed_data, cond_mask, observed_mask, side_info, is_train
    ):
        loss_sum = 0
        for t in range(self.num_steps):  # calculate loss for all t
            loss = self.calc_loss(
                observed_data, cond_mask, observed_mask, side_info, is_train, set_t=t
            )
            loss_sum += loss.detach()
        return loss_sum / self.num_steps

    """
    def calc_loss
       - Calculates loss for a specific step in the diffusion process during training or validation.
       - `is_train`: Flag to distinguish between training and validation modes.
       - `set_t`: Specific time step at which to calculate the loss; if not in training mode, the step is predetermined.
       - Determines the diffusion time step randomly during training or uses the provided step during validation.
       - Applies the diffusion model to compute predictions for noisy data, compares these predictions against actual noise added, 
         and computes the loss based on differences weighted by the target mask (areas not covered by the condition mask).
    """
    def calc_loss(
        self, observed_data, cond_mask, observed_mask, side_info, is_train, set_t=-1
    ):
        B, K, L = observed_data.shape
        if is_train != 1:  # for validation
            t = (torch.ones(B) * set_t).long().to(self.device)
        else:
            t = torch.randint(0, self.num_steps, [B]).to(self.device)
        current_alpha = self.alpha_torch[t]  # (B,1,1)
        noise = torch.randn_like(observed_data)
        noisy_data = (current_alpha ** 0.5) * observed_data + (1.0 - current_alpha) ** 0.5 * noise

        total_input = self.set_input_to_diffmodel(noisy_data, observed_data, cond_mask)

        predicted = self.diffmodel(total_input, side_info, t)  # (B,K,L)

        target_mask = observed_mask - cond_mask
        residual = (noise - predicted) * target_mask
        num_eval = target_mask.sum()
        loss = (residual ** 2).sum() / (num_eval if num_eval > 0 else 1)
        return loss

    """
    def set_input_to_diffmodel
       - Prepares the input tensors for the diffusion model based on the model's conditionality.
       - `noisy_data`: Tensor containing the data perturbed with noise.
       - `observed_data`: Original data tensor containing observed values.
       - `cond_mask`: A mask that distinguishes between observed and unobserved data points.
       - If the model is unconditional, it uses only the noisy data as input.
       - For conditional models, it creates separate channels for conditioned observed data and noisy targets, then concatenates them.
    """
    def set_input_to_diffmodel(self, noisy_data, observed_data, cond_mask):
        if self.is_unconditional == True:
            total_input = noisy_data.unsqueeze(1)  # (B,1,K,L)
        else:
            cond_obs = (cond_mask * observed_data).unsqueeze(1)
            noisy_target = ((1 - cond_mask) * noisy_data).unsqueeze(1)
            total_input = torch.cat([cond_obs, noisy_target], dim=1)  # (B,2,K,L)

        return total_input

    """
    def impute
       - Conducts the imputation of missing values based on observed data, condition masks, and side information.
       - `observed_data`: The observed portion of the data.
       - `cond_mask`: Mask indicating observed (1) and missing (0) parts of the data.
       - `side_info`: Contextual information that aids the model in imputation.
       - `n_samples`: Number of imputation samples to generate.
       - Initializes storage for the imputed samples and iteratively generates them using a diffusion process.
       - For unconditional models, creates noisy versions of the observed data for each diffusion step.
       - For conditional models, combines observed data and generated samples in a structured input for the diffusion model.
       - Utilizes backward diffusion (from the last step to the first) to progressively refine each imputed sample, adjusting with noise and model predictions.
       - Returns a tensor of imputed samples across all specified instances.
    """
    def impute(self, observed_data, cond_mask, side_info, n_samples):
        B, K, L = observed_data.shape

        imputed_samples = torch.zeros(B, n_samples, K, L).to(self.device)

        for i in range(n_samples):
            # generate noisy observation for unconditional model
            if self.is_unconditional == True:
                noisy_obs = observed_data
                noisy_cond_history = []
                for t in range(self.num_steps):
                    noise = torch.randn_like(noisy_obs)
                    noisy_obs = (self.alpha_hat[t] ** 0.5) * noisy_obs + self.beta[t] ** 0.5 * noise
                    noisy_cond_history.append(noisy_obs * cond_mask)

            current_sample = torch.randn_like(observed_data)

            for t in range(self.num_steps - 1, -1, -1):
                if self.is_unconditional == True:
                    diff_input = cond_mask * noisy_cond_history[t] + (1.0 - cond_mask) * current_sample
                    diff_input = diff_input.unsqueeze(1)  # (B,1,K,L)
                else:
                    cond_obs = (cond_mask * observed_data).unsqueeze(1)
                    noisy_target = ((1 - cond_mask) * current_sample).unsqueeze(1)
                    diff_input = torch.cat([cond_obs, noisy_target], dim=1)  # (B,2,K,L)
                predicted = self.diffmodel(diff_input, side_info, torch.tensor([t]).to(self.device))

                coeff1 = 1 / self.alpha_hat[t] ** 0.5
                coeff2 = (1 - self.alpha_hat[t]) / (1 - self.alpha[t]) ** 0.5
                current_sample = coeff1 * (current_sample - coeff2 * predicted)

                if t > 0:
                    noise = torch.randn_like(current_sample)
                    sigma = (
                        (1.0 - self.alpha[t - 1]) / (1.0 - self.alpha[t]) * self.beta[t]
                    ) ** 0.5
                    current_sample += sigma * noise

            imputed_samples[:, i] = current_sample.detach()
        return imputed_samples

    """
    def forward
       - Defines the forward pass of the model, integrating preprocessing, conditioning, and loss calculation.
       - `batch`: Input batch containing data and various masks.
       - `is_train`: Flag to indicate if the model is in training (1) or validation (0) mode.
       - Extracts observed data, masks, and time points from the batch.
       - Depending on the training mode and target strategy, determines the conditional mask for the data.
       - Computes side information based on observed time points and the conditional mask.
       - Selects the appropriate loss function based on the training mode.
       - Calculates and returns the loss, facilitating both training and validation processes.
    """
    def forward(self, batch, is_train=1):
        (
            observed_data,
            observed_mask,
            observed_tp,
            gt_mask,
            for_pattern_mask,
            _,
        ) = self.process_data(batch)
        if is_train == 0:
            cond_mask = gt_mask
        elif self.target_strategy != "random":
            cond_mask = self.get_hist_mask(
                observed_mask, for_pattern_mask=for_pattern_mask
            )
        else:
            cond_mask = self.get_randmask(observed_mask)

        side_info = self.get_side_info(observed_tp, cond_mask)

        loss_func = self.calc_loss if is_train == 1 else self.calc_loss_valid

        return loss_func(observed_data, cond_mask, observed_mask, side_info, is_train)

    """
    def evaluate
       - Conducts evaluation of the model by generating imputed samples and preparing metrics.
       - `batch`: Input batch containing observed data and masks.
       - `n_samples`: Number of imputation samples to generate for each point.
       - Processes the batch to extract data and masks, including the length for each sequence to avoid redundancy.
       - Sets up a non-gradient context for evaluation to prevent backpropagation and save computation.
       - Uses ground truth masks to determine conditional and target masks.
       - Retrieves side information based on observed time points and conditional masks.
       - Generates multiple imputed data samples.
       - Adjusts target masks to avoid evaluation on predefined cut lengths in data.
       - Returns the generated samples along with the observed data and masks for further assessment.
    """
    def evaluate(self, batch, n_samples):
        (
            observed_data,
            observed_mask,
            observed_tp,
            gt_mask,
            _,
            cut_length,
        ) = self.process_data(batch)

        with torch.no_grad():
            cond_mask = gt_mask
            target_mask = observed_mask - cond_mask

            side_info = self.get_side_info(observed_tp, cond_mask)

            samples = self.impute(observed_data, cond_mask, side_info, n_samples)

            for i in range(len(cut_length)):  # to avoid double evaluation
                target_mask[i, ..., 0 : cut_length[i].item()] = 0
        return samples, observed_data, target_mask, observed_mask, observed_tp

In [15]:
class CSDI_Forecasting(CSDI_base):
    """
     class CSDI_Forecasting Initializes the forecasting model derived from the CSDI_base class:
       - Sets up the model with specific configurations and assigns it to a computational device.
       - Initializes key attributes including the dimensionality of the target data and the number of sampling features based on the model configuration.
    """
    def __init__(self, config, device, target_dim):
        super(CSDI_Forecasting, self).__init__(target_dim, config, device)
        self.target_dim_base = target_dim
        self.num_sample_features = config["model"]["num_sample_features"]

    """
    def process_data
    - Processes the input batch for the model, formatting and preparing data for subsequent operations.
    - `batch`: The input batch containing observed data, masks, and timepoints.
    - Transfers all relevant data to the computation device and converts it to floating-point format.
    - Reorders dimensions of observed data and masks to match the expected input format of the model.
    - Initializes a tensor for cut lengths to dynamically manage sequence processing.
    - Generates a tensor of feature identifiers for subsequent data handling processes.
    """
    def process_data(self, batch):
        observed_data = batch["observed_data"].to(self.device).float()
        observed_mask = batch["observed_mask"].to(self.device).float()
        observed_tp = batch["timepoints"].to(self.device).float()
        gt_mask = batch["gt_mask"].to(self.device).float()

        observed_data = observed_data.permute(0, 2, 1)
        observed_mask = observed_mask.permute(0, 2, 1)
        gt_mask = gt_mask.permute(0, 2, 1)

        cut_length = torch.zeros(len(observed_data)).long().to(self.device)
        for_pattern_mask = observed_mask

        feature_id=torch.arange(self.target_dim_base).unsqueeze(0).expand(observed_data.shape[0],-1).to(self.device)

        return (
            observed_data,
            observed_mask,
            observed_tp,
            gt_mask,
            for_pattern_mask,
            cut_length,
            feature_id, 
        )  
          
    """
    def sample_features
    - Randomly samples a subset of features from the observed data for model processing.
    - `observed_data`: Tensor containing the observed data points.
    - `observed_mask`: Tensor indicating the presence of observed data points.
    - `feature_id`: Tensor of feature identifiers corresponding to observed data.
    - `gt_mask`: Tensor of ground truth masks indicating valid data points for evaluation.
    - Samples a specified number of features (defined by `num_sample_features`) from each observation in the batch.
    - Reorganizes the data, masks, and feature identifiers based on the sampled features to prepare for model input.
    - Returns the newly formatted data, masks, feature identifiers, and ground truth masks.
    """
    def sample_features(self,observed_data, observed_mask,feature_id,gt_mask):
        size = self.num_sample_features
        self.target_dim = size
        extracted_data = []
        extracted_mask = []
        extracted_feature_id = []
        extracted_gt_mask = []
        
        for k in range(len(observed_data)):
            ind = np.arange(self.target_dim_base)
            np.random.shuffle(ind)
            extracted_data.append(observed_data[k,ind[:size]])
            extracted_mask.append(observed_mask[k,ind[:size]])
            extracted_feature_id.append(feature_id[k,ind[:size]])
            extracted_gt_mask.append(gt_mask[k,ind[:size]])
        extracted_data = torch.stack(extracted_data,0)
        extracted_mask = torch.stack(extracted_mask,0)
        extracted_feature_id = torch.stack(extracted_feature_id,0)
        extracted_gt_mask = torch.stack(extracted_gt_mask,0)
        return extracted_data, extracted_mask,extracted_feature_id, extracted_gt_mask

    """
    def get_side_info
    - Generates side information combining time embeddings and feature embeddings based on provided masks.
    - `observed_tp`: Tensor containing the time points of observations.
    - `cond_mask`: Conditional mask tensor that specifies which data points are used for conditioning.
    - `feature_id`: Optional tensor of feature identifiers; used if model's target dimension differs from the base.
    - Constructs time embeddings for each time point and expands these across the target dimensions.
    - Depending on the model's configuration, either generates a static feature embedding for all features or specific embeddings based on the provided `feature_id`.
    - Concatenates time and feature embeddings to form a comprehensive side information tensor.
    - Reorders dimensions to match the expected input structure for further model processes.
    - If the model conditions on the input (is not unconditional), includes the conditional mask in the side information.
    - Returns the compiled side information for use in the model’s prediction or forecasting tasks.
    """
    def get_side_info(self, observed_tp, cond_mask,feature_id=None):
        B, K, L = cond_mask.shape

        time_embed = self.time_embedding(observed_tp, self.emb_time_dim)  # (B,L,emb)
        time_embed = time_embed.unsqueeze(2).expand(-1, -1, self.target_dim, -1)

        if self.target_dim == self.target_dim_base:
            feature_embed = self.embed_layer(
                torch.arange(self.target_dim).to(self.device)
            )  # (K,emb)
            feature_embed = feature_embed.unsqueeze(0).unsqueeze(0).expand(B, L, -1, -1)
        else:
            feature_embed = self.embed_layer(feature_id).unsqueeze(1).expand(-1,L,-1,-1)
        side_info = torch.cat([time_embed, feature_embed], dim=-1)  # (B,L,K,*)
        side_info = side_info.permute(0, 3, 2, 1)  # (B,*,K,L)

        if self.is_unconditional == False:
            side_mask = cond_mask.unsqueeze(1)  # (B,1,K,L)
            side_info = torch.cat([side_info, side_mask], dim=1)

        return side_info

    """
    def forward
    - Defines the forward pass for the model, processing input data and executing training or evaluation steps.
    - `batch`: Input batch containing observed data, masks, and additional information.
    - `is_train`: Indicator of whether the model is in training mode (1) or evaluation mode (0).
    - Processes the input batch to format and prepare data structures for model operations.
    - Conditionally samples a subset of features from the observed data if in training mode and feature sampling is necessary due to dimensionality constraints.
    - Sets the target dimensionality based on the model's base or sampled features.
    - Determines the conditioning mask based on the mode of operation, using ground truth masks for evaluation or generating test patterns for training.
    - Retrieves side information incorporating time and feature embeddings tailored to the conditioning context.
    - Selects the appropriate loss calculation function based on the training or evaluation context.
    - Computes and returns the loss by comparing model predictions with actual observations and side information, adjusted for the specified training or evaluation mode.
    """
    def forward(self, batch, is_train=1):
        (
            observed_data,
            observed_mask,
            observed_tp,
            gt_mask,
            _,
            _,
            feature_id, 
        ) = self.process_data(batch)
        if is_train == 1 and (self.target_dim_base > self.num_sample_features):
            observed_data, observed_mask,feature_id,gt_mask = \
                    self.sample_features(observed_data, observed_mask,feature_id,gt_mask)
        else:
            self.target_dim = self.target_dim_base
            feature_id = None

        if is_train == 0:
            cond_mask = gt_mask
        else: #test pattern
            cond_mask = self.get_test_pattern_mask(
                observed_mask, gt_mask
            )

        side_info = self.get_side_info(observed_tp, cond_mask, feature_id)

        loss_func = self.calc_loss if is_train == 1 else self.calc_loss_valid

        return loss_func(observed_data, cond_mask, observed_mask, side_info, is_train)

    """
    def evaluate
    - Conducts evaluation of the model by generating imputed samples and preparing metrics.
    - `batch`: Input batch containing observed data and masks.
    - `n_samples`: Number of imputation samples to generate for each point.
    - Processes the batch to extract data and masks, including the length for each sequence to avoid redundancy.
    - Sets up a non-gradient context for evaluation to prevent backpropagation and save computation.
    - Uses ground truth masks to determine conditional and target masks.
    - Retrieves side information based on observed time points and conditional masks.
    - Generates multiple imputed data samples.
    - Returns the generated samples along with the observed data and masks for further assessment.
    """
    def evaluate(self, batch, n_samples):
        (
            observed_data,
            observed_mask,
            observed_tp,
            gt_mask,
            _,
            _,
            feature_id, 
        ) = self.process_data(batch)

        with torch.no_grad():
            cond_mask = gt_mask
            target_mask = observed_mask * (1-gt_mask)

            side_info = self.get_side_info(observed_tp, cond_mask)

            samples = self.impute(observed_data, cond_mask, side_info, n_samples)

        return samples, observed_data, target_mask, observed_mask, observed_tp

## Experiment Setup and Execution

This segment highlights the setup and execution of utilizing CSDI for time series forecasting of electricity data. 

## Configuration and Argument Parsing
The experiment's settings are loaded from a YAML configuration file, allowing easy adjustments to the model and training parameters. Modifications to these settings via command line arguments are directly reflected in the configuration, ensuring that each experiment can be finely tuned.

In [25]:
"""{
    "train": {
        "epochs": 100,  # Total number of training cycles through the entire dataset.
        "batch_size": 8,  # Number of data samples processed before the model's internal parameters are updated.
        "lr": 0.001,  # Learning rate, determines the step size at each iteration while moving toward a minimum of the loss function.
        "itr_per_epoch": 100000000.0  # Presumably meant to be iterations per epoch.
    },
    "diffusion": {
        "layers": 4,  # Number of layers in the diffusion model, affects depth and complexity.
        "channels": 64,  # Number of channels in each layer, influences the model's capacity to process information.
        "nheads": 8,  # Number of attention heads in transformer-based models, affects the model's ability to focus on different parts of the input sequence.
        "diffusion_embedding_dim": 128,  # Dimension of the embeddings used in the diffusion process, impacts the representational power.
        "beta_start": 0.0001,  # Initial value of the noise schedule, dictates how much noise starts the diffusion process.
        "beta_end": 0.5,  # Final value in the noise schedule, controls how much noise is removed by the end of the diffusion process.
        "num_steps": 50,  # Total number of steps in the diffusion process from start to completion.
        "schedule": "quad",  # Type of scheduling for the beta values, 'quad' implies a quadratic progression.
        "is_linear": True  # Indicates whether the scheduling progression is linear, set to true which conflicts with 'quad' indicating a possible oversight or specific implementation.
    },
    "model": {
        "is_unconditional": 0,  # Specifies whether the model is unconditional (0 indicates conditional).
        "timeemb": 128,  # Dimension of the time embeddings, used in models that incorporate timing information in their predictions.
        "featureemb": 16,  # Dimension of the feature embeddings, provides additional contextual information per feature.
        "target_strategy": "test",  # Strategy for targeting in training/testing, 'test' might indicate a specific approach or mode used during evaluation.
        "num_sample_features": 64  # Number of features to sample, relevant in scenarios like feature ablation or when working with high-dimensional data.
    }
}
"""

def load_config(config_path='config/base_forecasting.yaml'):
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

config = load_config()
print(json.dumps(config, indent=4))

{
    "train": {
        "epochs": 100,
        "batch_size": 8,
        "lr": 0.001,
        "itr_per_epoch": 100000000.0
    },
    "diffusion": {
        "layers": 4,
        "channels": 64,
        "nheads": 8,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 50,
        "schedule": "quad",
        "is_linear": true
    },
    "model": {
        "is_unconditional": 0,
        "timeemb": 128,
        "featureemb": 16,
        "target_strategy": "test",
        "num_sample_features": 64
    }
}


## Model Setup and Data Loading

### Device Configuration
First, we determine the appropriate computation environment. If a GPU is available, the model will utilize it for faster computation; otherwise, it defaults to using the CPU. This ensures that the setup is optimized for performance regardless of the hardware available.

In [17]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Data Loaders
Data loaders are set up for the training, validation, and testing phases. These loaders are crucial for managing the data flow during model training and evaluation, ensuring efficient handling of data batches and the appropriate application of missing data simulations as specified in the experimental setup.

In [18]:
# Set up dataloaders
datatype = 'electricity'
target_dim = 370  # for electricity dataset

train_loader, valid_loader, test_loader, scaler, mean_scaler = get_dataloader(
    datatype=datatype,
    device=device,
    batch_size=config['train']['batch_size']
)

FileNotFoundError: [Errno 2] No such file or directory: './data/electricity_nips/data.pkl'

### Model Initialization
The model, `CSDI_Forecasting`, is initialized based on predefined configurations. This specialized model is designed to handle electricity time series data, inheriting robust functionalities from its base class to effectively manage the specific requirements of time series forecasting tasks.

In [21]:
# Set up model
model = CSDI_Forecasting(config, device, target_dim).to(device)

## Model Training

### Output Folder Setup
Before training the model, an output directory is created to store training artifacts such as model checkpoints, logs, and output files. The directory name includes a timestamp to ensure uniqueness and to help track experiments based on the date and time they were performed.

- **Directory Naming**: The folder is named using the current date and time, which helps in organizing and retrieving model training sessions based on when they were conducted.
- **Creation**: The directory is created on the file system, ensuring it exists before any training outputs are written to it. This prevents errors related to file writing during model training.

### Model Training Process
The model is trained using the specified configurations, data loaders, and the path to the output directory. The training function is designed to handle both the training and validation phases within each epoch, allowing for a comprehensive assessment of model performance over time.

- **Training Function**: Takes the model, training configurations, and data loaders as inputs. Additionally, it accepts the path to the output folder where the training results are stored.
- **Validation Data**: Optionally, a validation loader can be passed to periodically evaluate model performance on a separate validation set during the training process.

### Execution
Upon execution, the training process iteratively updates the model weights based on the loss computed from the training data. It also evaluates the model on the validation set, if provided, to monitor its performance on unseen data. Results and model states are saved in the designated output directory, facilitating post-training evaluations and model deployment.

### Loss Function
The loss function employed in the CSDI model is designed to optimize the model's ability to denoise data:
- **Denoising Loss**: During training, the model calculates the loss as the squared difference between the actual noise added to the data in the forward process and the noise predicted by the model during the reverse diffusion process. This loss function is key to training the model to accurately reverse the noise addition, effectively reconstructing the original data from its noisy version.

### Metrics
1. **Mean Absolute Error (MAE)**: This metric measures the average magnitude of errors in a set of predictions, without considering their direction. It's a linear score that averages the absolute differences between predicted and actual values, providing a straightforward interpretation of prediction accuracy.
2. **Continuous Ranked Probability Score (CRPS)**: CRPS is used to assess the accuracy of probabilistic predictions. It measures the difference between the predicted cumulative distribution function and the empirical distribution function of the observed data. This score is particularly useful for evaluating the performance of models that generate probabilistic or distributional forecasts.

In [22]:
# Set up output folder
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
foldername = f"./save/forecasting_{datatype}_{current_time}/"
os.makedirs(foldername, exist_ok=True)

# Save config
with open(foldername + "config.json", "w") as f:
    json.dump(config, f, indent=4)

In [7]:
# Train the model
train(
    model,
    config['train'],
    train_loader,
    valid_loader=valid_loader,
    foldername=foldername
)

  3%|█▌                                                          | 18/691 [00:55<34:45,  3.10s/it, avg_epoch_loss=0.658, epoch=0]


KeyboardInterrupt: 

## Loading a Pretrained Model

### Function for Loading Model State
To enhance or expedite the training process, or for evaluation purposes, you may start with a model that has already been trained. The function `load_pretrained_model` facilitates the loading of these pretrained weights into an existing model architecture.

- **Parameters**:
  - `model`: The model instance into which the pretrained weights will be loaded.
  - `modelfolder`: The subdirectory under `./save/` where the pretrained model is stored, defaulting to `pretrained`.
  - `device`: The computing device (CPU or GPU) where the model will be loaded. This ensures that the model is compatible with the hardware used for subsequent operations.

### Execution
The function constructs the full path to the pretrained model's state dictionary file (`model.pth`) using the specified `modelfolder`. It then loads this state dictionary into the model, ensuring that all model parameters are updated accordingly.

- **Model Compatibility**:
  - It is crucial that the model architecture into which the weights are being loaded matches the architecture of the model when it was saved. Incompatibility in architectures will lead to errors during the loading process.

### Usage
To utilize a pretrained model, simply pass your initialized but untrained model to the `load_pretrained_model` function. This setup allows you to leverage previously learned patterns, potentially reducing training time and improving model robustness from the outset.

In [23]:
def load_pretrained_model(model, modelfolder='pretrained', device=device):
    model_path = f"./save/{modelfolder}/model.pth"
    model.load_state_dict(torch.load(model_path, map_location=device))
    return model

# Load pre-trained model
pretrained_model = load_pretrained_model(model)
pretrained_model.target_dim = target_dim

RuntimeError: Error(s) in loading state_dict for CSDI_Forecasting:
	Missing key(s) in state_dict: "diffmodel.residual_layers.0.time_layer.layers.layers.0.0.fn.to_q.weight", "diffmodel.residual_layers.0.time_layer.layers.layers.0.0.fn.to_k.weight", "diffmodel.residual_layers.0.time_layer.layers.layers.0.0.fn.to_v.weight", "diffmodel.residual_layers.0.time_layer.layers.layers.0.0.fn.to_out.weight", "diffmodel.residual_layers.0.time_layer.layers.layers.0.0.fn.to_out.bias", "diffmodel.residual_layers.0.time_layer.layers.layers.0.0.norm.weight", "diffmodel.residual_layers.0.time_layer.layers.layers.0.0.norm.bias", "diffmodel.residual_layers.0.time_layer.layers.layers.0.1.fn.fn.w1.weight", "diffmodel.residual_layers.0.time_layer.layers.layers.0.1.fn.fn.w1.bias", "diffmodel.residual_layers.0.time_layer.layers.layers.0.1.fn.fn.w2.weight", "diffmodel.residual_layers.0.time_layer.layers.layers.0.1.fn.fn.w2.bias", "diffmodel.residual_layers.0.time_layer.layers.layers.0.1.norm.weight", "diffmodel.

## Model Evaluation

### Evaluation Parameters Configuration
Prior to evaluating the models, several key parameters are established:
- **`test_loader`**: The data loader for the test dataset.
- **`nsample`**: Specifies the number of samples to generate during model evaluation, set to 100 for comprehensive testing.
- **`scaler`** and **`mean_scaler`**: Define scaling factors for the data. These parameters adjust the data normalization during the evaluation to match the conditions used during model training.

### Data Loader Configuration
- The test data loader is updated to include the new missing ratio, ensuring that the evaluation tests the model's ability to handle and impute missing data effectively.

### Model Evaluation
- **Current Model Evaluation**: The initially trained model is evaluated using the updated `test_loader`. This step is crucial for understanding the baseline performance of the model on the test set.
- **Pre-trained Model Evaluation**: Additionally, a pre-trained model is evaluated under the same conditions. This is particularly useful for comparing the effectiveness of pre-training and fine-tuning strategies on model performance.

### Execution
Both the current and pre-trained models are evaluated with the specified number of samples and scaling parameters. The results are stored in a designated folder, facilitating subsequent analysis and comparison:

In [ ]:
# Set evaluation parameters
nsample = 100  # number of samples for evaluation

# Evaluate the model
evaluate(
    model,
    test_loader,
    nsample=nsample,
    scaler=scaler,
    mean_scaler=mean_scaler,
    foldername=foldername
)

### References
- Tashiro, Yusuke, et al. "CSDI: Conditional Score-based Diffusion Models for Probabilistic Time Series Imputation." *Advances in Neural Information Processing Systems*. 2021. [GitHub Repository](https://github.com/ermongroup/CSDI)
